In [15]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [16]:
df = spark.read.parquet('domain2016-2021/*')

In [17]:
df = df.withColumnRenamed('price','price_sold')

In [18]:
df.printSchema()

root
 |-- price_sold: string (nullable = true)
 |-- date_sold: date (nullable = true)
 |-- suburb: string (nullable = true)
 |-- num_bath: integer (nullable = true)
 |-- num_bed: integer (nullable = true)
 |-- num_parking: integer (nullable = true)
 |-- property_size: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- suburb_population: integer (nullable = true)
 |-- suburb_median_income: integer (nullable = true)
 |-- suburb_sqkm: double (nullable = true)
 |-- suburb_lat: double (nullable = true)
 |-- suburb_lng: double (nullable = true)
 |-- suburb_elevation: integer (nullable = true)
 |-- cash_rate: double (nullable = true)
 |-- property_inflation_index: double (nullable = true)
 |-- km_from_cbd: double (nullable = true)



In [19]:
df.registerTempTable('domain')

In [20]:
spark.sql("""
SELECT
    type,
    count(1)
FROM
    domain
GROUP BY 
    type
""").show()

+--------------------+--------+
|                type|count(1)|
+--------------------+--------+
|           Townhouse|     211|
|       Semi-Detached|     170|
|              Studio|       5|
|            New land|       3|
|               Villa|     114|
|    Development Site|       7|
|Apartment / Unit ...|     688|
|               House|    9583|
|New Apartments / ...|       9|
|      Block of Units|      37|
|    New House & Land|      15|
|             Terrace|      63|
|         Vacant land|     163|
|Acreage / Semi-Rural|      21|
|              Duplex|      67|
|               Rural|       4|
+--------------------+--------+



In [24]:
df_result = spark.sql("""
SELECT 
    -- Grouping 
    suburb,
    type,
    date_trunc('month', date_sold) AS month_sold,
 
    -- Aggregation 
    AVG(price_sold) AS avg_price_sold,
    count(1) AS quantity_sold,
    AVG(suburb_population) AS suburb_population,
    AVG(suburb_median_income) AS suburb_median_income,
    AVG(cash_rate) AS cash_rate

FROM
    domain
GROUP BY
    1, 2, 3
""")

In [25]:
df_result.show()

+------------------+--------------------+-------------------+------------------+-------------+-----------------+--------------------+-------------------+
|            suburb|                type|         month_sold|    avg_price_sold|quantity_sold|suburb_population|suburb_median_income|          cash_rate|
+------------------+--------------------+-------------------+------------------+-------------+-----------------+--------------------+-------------------+
|      Melrose Park|               House|2016-10-01 00:00:00|         2620000.0|            1|           1574.0|             41028.0|                1.5|
|         Girraween|               House|2018-04-01 00:00:00|          695000.0|            1|           5590.0|             35152.0|                1.5|
|             Menai|               House|2021-10-01 00:00:00|         1567000.0|            1|          10304.0|             43836.0|                0.1|
|            Sydney|Apartment / Unit ...|2016-12-01 00:00:00|         202000

In [26]:
df_result.coalesce(1).write.parquet('domain2016-2021/report/',mode='overwrite')